In [ ]:
!pip install econml

: 

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from econml.dml import DML
from math import sqrt

# --- Step 1: Load dataset ---
dataset_path = "/content/sample_data/main.csv"   # Change if needed
df = pd.read_csv(dataset_path)

# --- Step 2: Define features ---
weather_features = [
    "temperature_2m (°C)",
    "relative_humidity_2m (%)",
    "rain (mm)",
    "wind_speed_100m (km/h)",
    "wind_direction_100m (°)",
    "pressure_msl (hPa)",
    "surface_pressure (hPa)"
]

pollutant_features = ["co", "no", "no2", "o3", "so2", "pm2_5", "pm10", "nh3"]

T = df[weather_features].values        # Treatments (inputs)
Y = df[pollutant_features].values      # Outcomes (outputs)

# --- Step 3: Train-test split (80/20) ---
X_train, X_test, T_train, T_test, Y_train, Y_test = train_test_split(
    df[weather_features], T, Y, test_size=0.2, random_state=42
)

# --- Step 4: Define models ---
model_y = RandomForestRegressor(n_estimators=200, max_depth=5, random_state=42)
model_t = RandomForestRegressor(n_estimators=200, max_depth=5, random_state=42)
model_final = RandomForestRegressor(n_estimators=200, max_depth=5, random_state=42)

# --- Step 5: Initialize and fit DML ---
dml = DML(
    model_y=model_y,
    model_t=model_t,
    model_final=model_final,
    discrete_treatment=False,
    random_state=42
)

dml.fit(Y_train, T_train, X=X_train)

# --- Step 6: Estimate treatment effects ---
te_pred = dml.const_marginal_effect(X_test)  # shape: (n_samples, n_treatments, n_outcomes)
print("Predicted treatment effects shape:", te_pred.shape)
print("First 3 treatment effect samples:")
print(te_pred[:3])

# Example: Effect of temperature (1st treatment) on pollutants
te_temp = te_pred[:, 0, :]  # (n_samples, n_outcomes)
print("\nEffect of temperature on pollutants (first 3 samples):")
print(te_temp[:3])

# --- Step 7: Train a standalone predictor for evaluation ---
# (DML doesn’t expose a fitted outcome predictor, so we fit one separately)
rf_outcome = RandomForestRegressor(n_estimators=200, max_depth=5, random_state=42)
rf_outcome.fit(X_train, Y_train)
Y_pred = rf_outcome.predict(X_test)



print("\nRMSE for each pollutant:")
for i, pollutant in enumerate(pollutant_features):
    mse = mean_squared_error(Y_test[:, i], Y_pred[:, i])  # no squared arg
    rmse = sqrt(mse)
    print(f"{pollutant}: {rmse:.3f}")



/usr/local/lib/python3.12/dist-packages/econml/dml/dml.py:192: UserWarning: The final model has a nonzero intercept for at least one outcome; it will be subtracted, but consider fitting a model without an intercept if possible.
  warn("The final model has a nonzero intercept for at least one outcome; "


Predicted treatment effects shape: (3756, 8, 7)
First 3 treatment effect samples:
[[[-2.17821472e+01 -7.66181818e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [-3.22944450e-01 -7.57866682e-02  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [-1.47513670e-01 -1.43629846e-01  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 1.29670840e-01 -6.19866393e-02  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [-9.43838405e-02 -3.95270514e-02  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [-1.95768615e+00 -3.81499117e-01  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [-2.31456598e+00 -6.40759478e-01  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [-2.59195449e-01 -1.22605102e-01  0.00000000e+00  0.00000000e+00
    0.00000000e+00